Necessary Imports

In [41]:
import pandas as pd
import re
from sklearn import svm
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# model = Perceptron()
# model = svm.SVC()
model = KNeighborsClassifier(n_neighbors=5)
# model = GaussianNB()

Read in Data, drop certain cells from database and turn all fields into numbers

In [42]:
# Read in Data
db = pd.read_csv("countries.csv", encoding='latin-1')

# Drop columns from database
to_drop = ["CountryID", "Country Name", "WEBNAME", "Country"]
db = db.drop(to_drop, axis=1)

# Drop rows with na values
db = db.dropna()

# Turns regions into indices
REGIONS = ["Asia-Pacific", "Europe", "Middle East and North Africa", "Sub-Saharan Africa", "Americas"]
db.Region = db.Region.apply(REGIONS.index)

# Turns values into floats:
def floatify(st):
    try:
        return float(st)
    except:
        if st[0] == "$":
            st = st[1:]
        st = st.replace(",", "")
        return float(st)

for col in db.columns:
    db[col] = db[col].apply(floatify)

# Add a column for unemployment quartile (1-4)
quartiles = []
for value in db["Unemployment (%)"]:
    if value <= 3.7:
        quartiles.append(1)
    elif value <= 5.55:
        quartiles.append(2)
    elif value <= 9.3:
        quartiles.append(3)
    else:
        quartiles.append(4)
        
db["Unemployment Quartile"] = quartiles

db = db.drop(["Unemployment (%)"], axis=1)


Turn dataframe into lists of evidence and labels

In [43]:
evidence = db.drop(["Unemployment Quartile"], axis=1)
labels = db[["Unemployment Quartile"]]

evidence = evidence.values.tolist()
labels = [ls[0] for ls in labels.values.tolist()]

Split into training/testing and train the model

In [44]:
X_training, X_testing, y_training, y_testing = train_test_split(
    evidence, labels, test_size=0.2
)

# Fir the model:
model.fit(X_training, y_training)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

Predicting and testing

In [45]:
# Make predictions on the testing set
predictions = model.predict(X_testing)

# Compute how well we performed
correct = (y_testing == predictions).sum()
incorrect = (y_testing != predictions).sum()
total = len(predictions)

# Print results
print(f"Results for model {type(model).__name__}")
print(f"Correct: {correct}")
print(f"Incorrect: {incorrect}")
print(f"Accuracy: {100 * correct / total:.2f}%")

Results for model KNeighborsClassifier
Correct: 8
Incorrect: 27
Accuracy: 22.86%
